### Monthly Income (based on pop & housing density)
*Split by homeowners & renters

In [1]:
import pandas as pd
import numpy as np

In [2]:
%run chart_theme.py

In [3]:
renter_inc_by_hous_den5 = pd.read_csv('renter_inc_by_hous_den_5yr_0816.csv')
renter_inc_by_hous_den1 = pd.read_csv('renter_inc_by_hous_den_1yr_1018.csv')

In [4]:
renter_inc_hous_den = pd.concat([renter_inc_by_hous_den5, renter_inc_by_hous_den1[renter_inc_by_hous_den1.Year > 2016]]).sort_values(['Hous_Den', 'Year']).reset_index(drop=True)

In [5]:
renter_inc_hous_den.head(1)

,Year,Hous_Den,Med_Rent
0,2008,10 to 20,1958.0


*Adjusting 2017 & 2018 values for '10-25' & 'Less than 10' cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [6]:
renter_inc_hous_den['pct_chg'] = renter_inc_hous_den['Med_Rent'].pct_change()

In [7]:
for year in [2017, 2018]:
    yr_df = renter_inc_hous_den[renter_inc_hous_den.Year == year]
    for typ in ['30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
        index = yr_df[yr_df.Hous_Den == typ].index[0]
        renter_inc_hous_den.loc[index, 'Med_Rent'] = round(renter_inc_hous_den['Med_Rent'][(renter_inc_hous_den.Year == year-1) & (renter_inc_hous_den.Hous_Den == typ)].iloc[0] + renter_inc_hous_den['Med_Rent'][(renter_inc_hous_den.Year == year-1) & (renter_inc_hous_den.Hous_Den == typ)].iloc[0] * renter_inc_hous_den.pct_chg[(renter_inc_hous_den.Year == year) & (renter_inc_hous_den.Hous_Den == 'More than 75')].iloc[0], 3)

In [8]:
dic = {}
for t in ['More than 75', '30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
    dic[t] = t.split()[-1]

In [9]:
renter_inc_hous_den['Order'] = renter_inc_hous_den.Hous_Den.map(dic).astype(int)

In [10]:
renter_inc_hous_den_df = renter_inc_hous_den.sort_values(['Order', 'Hous_Den'], ascending=[False, False])

In [65]:
%run chart_theme.py

In [11]:
"""rent_inc_hous_den = alt.Chart(renter_inc_hous_den_df, title='Monthly Renter Income').mark_line(size=9, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=26, labelOpacity=.7)),
y=alt.Y('Med_Rent', title='Counties grouped by housing density', 
        scale=alt.Scale(domain=[1000,8000]),
        axis=alt.Axis(values=list(range(2000,8000,1000)), format='~s',
                      titleX=0, titleY=-30, titleFontSize=28, titleOpacity=.7,
                      labelFontSize=26, labelOpacity=0, labelPadding=0)),
color=alt.Color('Hous_Den', scale=alt.Scale(domain=list(renter_inc_hous_den_df.Hous_Den.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#d1e5f0','#67a9cf','#2166ac']), 
                title='Housing Units / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[8000], 'text':['$8k per month']})
y_text = alt.Chart(y_text_df).mark_text(dx=70, dy=0, fontSize=26, font='lato', opacity=0).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[8000], 'text':['_'*94]})).mark_text(dx=275, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


rent_inc_hous_den_chart = (rent_inc_hous_den + y_text).properties(width=600, height=450)"""

In [76]:
rent_inc_hous_den = alt.Chart(renter_inc_hous_den_df, title='Monthly Renter Income').mark_line(size=9, opacity=1, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=28, labelFontSize=26, labelOpacity=.8)),
y=alt.Y('Med_Rent', title='Counties grouped by housing density', 
        scale=alt.Scale(domain=[1000,7000]),
        axis=alt.Axis(values=list(range(2000,7000,1000)), format='~s',
                      titleX=-25, titleY=-30, titleFontSize=28, titleOpacity=.8,
                      labelFontSize=26, labelOpacity=.8, labelPadding=25)),
color=alt.Color('Hous_Den', scale=alt.Scale(domain=list(renter_inc_hous_den_df.Hous_Den.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#d1e5f0','#67a9cf','#2166ac']), 
                title='Housing Units / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[7000], 'text':['$7k per month']})
y_text = alt.Chart(y_text_df).mark_text(dx=32, dy=0, fontSize=26, font='lato', opacity=.8).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[7000], 'text':['_'*72]})).mark_text(dx=345, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


rent_inc_hous_den_chart = (rent_inc_hous_den + y_text + line).properties(width=600, height=450)

In [77]:
rent_inc_hous_den_chart

alt.LayerChart(...)

In [21]:
rent_hous_den_chart.save('rent_hous_den.png', scale_factor=3)

---

In [13]:
homeowner_inc_by_hous_den5 = pd.read_csv('homeowner_inc_by_hous_den_5yr_0816.csv')
homeowner_inc_by_hous_den1 = pd.read_csv('homeowner_inc_by_hous_den_1yr_1018.csv')

In [14]:
homeowner_inc_hous_den_df = pd.concat([homeowner_inc_by_hous_den5, homeowner_inc_by_hous_den1[homeowner_inc_by_hous_den1.Year > 2016]]).sort_values(['Hous_Den', 'Year']).reset_index(drop=True)

In [15]:
homeowner_inc_hous_den_df.head(1)

,Year,Hous_Den,Med_Mort
0,2008,10 to 20,4051.0


*Adjusting 2017 & 2018 values for '10-25' & 'Less than 10' cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [16]:
homeowner_inc_hous_den_df['pct_chg'] = homeowner_inc_hous_den_df['Med_Mort'].pct_change()

In [17]:
for year in [2017, 2018]:
    yr_df = homeowner_inc_hous_den_df[homeowner_inc_hous_den_df.Year == year]
    for typ in ['30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
        index = yr_df[yr_df.Hous_Den == typ].index[0]
        homeowner_inc_hous_den_df.loc[index, 'Med_Mort'] = round(homeowner_inc_hous_den_df['Med_Mort'][(homeowner_inc_hous_den_df.Year == year-1) & (homeowner_inc_hous_den_df.Hous_Den == typ)].iloc[0] + homeowner_inc_hous_den_df['Med_Mort'][(homeowner_inc_hous_den_df.Year == year-1) & (homeowner_inc_hous_den_df.Hous_Den == typ)].iloc[0] * homeowner_inc_hous_den_df.pct_chg[(homeowner_inc_hous_den_df.Year == year) & (homeowner_inc_hous_den_df.Hous_Den == 'More than 75')].iloc[0], 3)

In [18]:
dic = {}
for t in ['More than 75', '30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
    dic[t] = t.split()[-1]

In [19]:
homeowner_inc_hous_den_df['Order'] = homeowner_inc_hous_den_df.Hous_Den.map(dic).astype(int)

In [20]:
homeowner_inc_hous_den_df = homeowner_inc_hous_den_df.sort_values(['Order', 'Hous_Den'], ascending=[False, False])

In [68]:
homeowner_inc_hous_den = alt.Chart(homeowner_inc_hous_den_df, title='Monthly Homeowner Income').mark_line(size=9, opacity=1, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=28, labelFontSize=26, labelOpacity=.8)),
y=alt.Y('Med_Mort', title='Counties grouped by housing density', 
        scale=alt.Scale(domain=[1000, 7000]),
        axis=alt.Axis(values=list(range(2000, 7000, 1000)), format='~s',
                      titleX=-2, titleY=-30, titleFontSize=28, titleOpacity=.8,
                      labelFontSize=26, labelOpacity=0, labelPadding=0)),
color=alt.Color('Hous_Den', scale=alt.Scale(domain=list(homeowner_inc_hous_den_df.Hous_Den.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#d1e5f0','#67a9cf','#2166ac']), 
                title='Housing Units / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[7000], 'text':['$7k per month']})
y_text = alt.Chart(y_text_df).mark_text(dx=70, dy=0, fontSize=26, font='lato', opacity=.8).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[7000], 'text':['_'*94]})).mark_text(dx=275, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


homeowner_inc_hous_den_chart = (homeowner_inc_hous_den + line).properties(width=600, height=450)

In [69]:
comb_hous_den_chart = (rent_inc_hous_den_chart | homeowner_inc_hous_den_chart).configure_legend(titleFont='Lato', titleFontSize=26, titleOpacity=.85, titleLimit=300, labelFont='Lato', labelFontSize=26, labelOpacity=.85, symbolSize=400, symbolStrokeWidth=9)
comb_hous_den_chart

alt.HConcatChart(...)

In [70]:
comb_hous_den_chart.save('comb_hous_den.svg')

In [121]:
# Renters & Homeowner lines on same chart (looks too busy)
(homeowner_inc_hous_den_chart + rent_inc_hous_den_chart).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

alt.LayerChart(...)

---

In [31]:
renter_inc_by_pop_den5 = pd.read_csv('renter_inc_by_pop_den_5yr_0816.csv')
renter_inc_by_pop_den1 = pd.read_csv('renter_inc_by_pop_den_1yr_1018.csv')

In [34]:
renter_inc_pop_den_df = pd.concat([renter_inc_by_pop_den5, renter_inc_by_pop_den1[renter_inc_by_pop_den1.Year > 2016]]).sort_values(['Pop', 'Year']).reset_index(drop=True)

In [49]:
renter_inc_pop_den_df.head(1)

,Year,Pop,Med_Rent
0,2008,10 to 25,2124.0


*Adjusting 2017 & 2018 values for '10-25' & 'Less than 10' cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [51]:
renter_inc_pop_den_df['pct_chg'] = renter_inc_pop_den_df['Med_Rent'].pct_change()

In [52]:
for year in [2017, 2018]:
    yr_df = renter_inc_pop_den_df[renter_inc_pop_den_df.Year == year]
    for typ in ['250 to 1000', '100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
        index = yr_df[yr_df.Pop == typ].index[0]
        renter_inc_pop_den_df.loc[index, 'Med_Rent'] = round(renter_inc_pop_den_df['Med_Rent'][(renter_inc_pop_den_df.Year == year-1) & (renter_inc_pop_den_df.Pop == typ)].iloc[0] + renter_inc_pop_den_df['Med_Rent'][(renter_inc_pop_den_df.Year == year-1) & (renter_inc_pop_den_df.Pop == typ)].iloc[0] * renter_inc_pop_den_df.pct_chg[(renter_inc_pop_den_df.Year == year) & (renter_inc_pop_den_df.Pop == 'More than 1000')].iloc[0], 3)

In [53]:
dic = {}
for t in ['More than 1000', '250 to 1000', '100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
    dic[t] = t.split()[-1]

In [54]:
renter_inc_pop_den_df['Order'] = renter_inc_pop_den_df.Pop.map(dic).astype(int)

In [55]:
renter_inc_pop_den_df = renter_inc_pop_den_df.sort_values(['Order', 'Pop'], ascending=[False, False])

In [107]:
rent_inc_pop_den = alt.Chart(renter_inc_pop_den_df, title='Monthly Renter Income').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('Med_Rent', title='Counties grouped by population density', 
        scale=alt.Scale(domain=[1000,8000, 2000]),
        axis=alt.Axis(values=list(range(2000, 8000, 1000)), format='~s', 
                      titleX=-25, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=25)),
color=alt.Color('Pop', scale=alt.Scale(domain=list(renter_inc_pop_den_df.Pop.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#e2e0d3', '#d1e5f0','#67a9cf','#2166ac']), 
                title='Residents / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[8000], 'text':['$8k per month']})
y_text = alt.Chart(y_text_df).mark_text(dx=8, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[8000], 'text':['_'*79]})).mark_text(dx=322, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


rent_inc_pop_den_chart = (rent_inc_pop_den + y_text + line).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

In [108]:
rent_inc_pop_den_chart

alt.LayerChart(...)

---

In [84]:
homeowner_inc_by_pop_den5 = pd.read_csv('homeowner_inc_by_pop_den_5yr_0816.csv')
homeowner_inc_by_pop_den1 = pd.read_csv('homeowner_inc_by_pop_den_1yr_1018.csv')

In [85]:
homeowner_inc_pop_den_df = pd.concat([homeowner_inc_by_pop_den5, homeowner_inc_by_pop_den1[homeowner_inc_by_pop_den1.Year > 2016]]).sort_values(['Pop', 'Year']).reset_index(drop=True)

In [86]:
homeowner_inc_pop_den_df.tail(1)

,Year,Pop,Med_Mort
76,2018,More than 1000,7767.0


*Adjusting 2017 & 2018 values for '10-25' & 'Less than 10' cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [87]:
homeowner_inc_pop_den_df['pct_chg'] = homeowner_inc_pop_den_df['Med_Mort'].pct_change()

In [88]:
for year in [2017, 2018]:
    yr_df = homeowner_inc_pop_den_df[homeowner_inc_pop_den_df.Year == year]
    for typ in ['250 to 1000', '100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
        index = yr_df[yr_df.Pop == typ].index[0]
        homeowner_inc_pop_den_df.loc[index, 'Med_Mort'] = round(homeowner_inc_pop_den_df['Med_Mort'][(homeowner_inc_pop_den_df.Year == year-1) & (homeowner_inc_pop_den_df.Pop == typ)].iloc[0] + homeowner_inc_pop_den_df['Med_Mort'][(homeowner_inc_pop_den_df.Year == year-1) & (homeowner_inc_pop_den_df.Pop == typ)].iloc[0] * homeowner_inc_pop_den_df.pct_chg[(homeowner_inc_pop_den_df.Year == year) & (homeowner_inc_pop_den_df.Pop == 'More than 1000')].iloc[0], 3)

In [89]:
dic = {}
for t in ['More than 1000', '250 to 1000', '100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
    dic[t] = t.split()[-1]

In [90]:
homeowner_inc_pop_den_df['Order'] = homeowner_inc_pop_den_df.Pop.map(dic).astype(int)

In [91]:
homeowner_inc_pop_den_df = homeowner_inc_pop_den_df.sort_values(['Order', 'Pop'], ascending=[False, False])

In [95]:
homeowner_inc_pop_den = alt.Chart(homeowner_inc_pop_den_df, title='Monthly Homeowner Income').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('Med_Mort', title='Counties grouped by population density', 
        scale=alt.Scale(domain=[2000,8000]),
        axis=alt.Axis(values=list(range(2000, 8000, 1000)), format='~s', 
                      titleX=-25, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=25)),
color=alt.Color('Pop', scale=alt.Scale(domain=list(homeowner_inc_pop_den_df.Pop.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#e2e0d3', '#d1e5f0','#67a9cf','#2166ac']), 
                title='Residents / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[8000], 'text':['$8k per month']})
y_text = alt.Chart(y_text_df).mark_text(dx=8, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[8000], 'text':['_'*79]})).mark_text(dx=322, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


homeowner_inc_pop_den_chart = (homeowner_inc_pop_den + y_text + line).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

In [96]:
homeowner_inc_pop_den_chart

alt.LayerChart(...)

---